In [ ]:
import os
from glob import glob
import numpy as np
import pickle
import matplotlib.pyplot as plt
plt.style.use(os.path.join('..', 'configs', 'mplstyle.yaml'))

from lib import utils, app
from lib.misc import get_markers
from lib.points import get_pairwise_3d_points_from_df

%load_ext autoreload
%autoreload 2

ROOT_DATA_DIR = os.path.join("..", "data")

# Define the params in the cell below. Thereafter, run all!

In [ ]:
DATA_DIR = os.path.join(ROOT_DATA_DIR, "2017_08_29 - Copy", "top", "jules", "run1_1")

start_frame = 20
end_frame = 70

# DLC p_cutoff - any points with likelihood < dlc_thresh are not trusted in EKF
dlc_thresh = 0.7  # change this only if FTE result is unsatisfactory

# SBA

## Function definitions

In [ ]:
def save_sba(positions, path, start_frame, dlc_thresh):
    import scipy.io
    types = ["", "_padded"]
    sba_path = os.path.join(path, "sba")
    os.makedirs(sba_path, exist_ok=True)
    
    for t in types:
        
        if "padded" in t:
            nan_arr = np.full((start_frame, positions.shape[1],3), np.nan)
            positions = np.concatenate([nan_arr, positions])
        
        file_data = dict(positions=positions)

        with open(os.path.join(sba_path, f"sba{t}.pickle"), 'wb') as f:
            pickle.dump(file_data, f)
        
        scipy.io.savemat(os.path.join(sba_path, f'sba{t}.mat'), file_data)
        
        if "padded" in t:
            out_fpath = os.path.join(sba_path, "sba.avi")
            app.reconstruction_reprojection_video(path, out_fpath, positions, include_lure=True, dlc_thresh=dlc_thresh)
        
    print('\nSaved files to', sba_path)

## Perform SBA

In [ ]:
start_frame -= 1
N = end_frame-start_frame

*_, n_cams, scene_fpath = utils.find_scene_file(DATA_DIR)

dlc_points_fpaths = glob(os.path.join(DATA_DIR, '*.h5'))
assert n_cams == len(dlc_points_fpaths)
    
# Load Measurement Data (pixels, likelihood)
points_2d_df = utils.load_dlc_points_as_df(dlc_points_fpaths)
points_2d_df = points_2d_df[points_2d_df["frame"].between(start_frame, end_frame-1)]
points_2d_df = points_2d_df[points_2d_df['likelihood']>dlc_thresh] # ignore points with low likelihood
# print("DLC points dataframe:\n", points_2d_df)

points_3d_df, residuals = app.sba_points_fisheye(scene_fpath, points_2d_df)
plt.plot(residuals['before'], label="Cost before")
plt.plot(residuals['after'], label="Cost after")
plt.legend()
plt.show()

In [ ]:
markers = get_markers(include_lure=True)

sba_positions = np.full((N,len(markers),3), np.nan)
for i, marker in enumerate(markers):
    marker_pts = points_3d_df[points_3d_df["marker"]==marker][["frame", "x", "y", "z"]].values
    for frame, *pt_3d in marker_pts:
        sba_positions[int(frame)-start_frame, i] = pt_3d
        
save_sba(np.array(sba_positions), DATA_DIR, start_frame, dlc_thresh)

## Plot the result!

In [ ]:
with open(os.path.join(DATA_DIR, "sba", 'sba.pickle'), 'rb') as f:
    sba_data = pickle.load(f)
sba_positions = sba_data["positions"]

ca = plotting.Cheetah(sba_positions, DATA_DIR, reprojections=True, dark_mode=True)
ca.animation()